<a href="https://colab.research.google.com/github/Winston68432/streamlit-example/blob/master/Copie_de_Copie_de_NBA_(11).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install -q streamlit




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:

!pip install -q streamlit_option_menu 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.3/712.3 KB 12.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df_nba_complete = pd.read_csv("NBA Shot Locations 1997 - 2020.csv")
df_player = pd.read_csv("player_data.csv",sep = ',')

# reduction aux 20 joueurs
names = ['Tim Duncan','Kobe Bryant','Allen Iverson','Steve Nash','Ray Allen','Paul Pierce','Pau Gasol','Tony Parker','Manu Ginobili','Dwyane Wade',
   'LeBron James','Chris Paul','Kevin Durant','Russell Westbrook','James Harden','Stephen Curry','Kawhi Leonard','Damian Lillard','Anthony Davis','Giannis Antetokounmpo']
df_nba=df_nba_complete[df_nba_complete['Player Name'].isin(names)]
# st.image("nba_project.PNG")

#Merge avec df_player
df_nba=df_nba.merge(df_player,how='left',left_on="Player Name", right_on="name")

#Ajoute shottime
df_nba.loc[df_nba['Period'] < 5, 'Shot_Time'] = (720-(60 * df_nba['Minutes Remaining'] + df_nba['Seconds Remaining']) + (df_nba['Period']-1)*12*60)
df_nba.loc[df_nba['Period'] > 4, 'Shot_Time'] = (300-(60 * df_nba['Minutes Remaining'] +df_nba['Seconds Remaining']) + ((df_nba['Period']-5)*5+48)*60)

df_nba['Shot_Time'] = df_nba['Shot_Time'].round(decimals=0).astype(int)

#clustering en fonction des actions réussis
df_nba.loc[df_nba['Shot Made Flag'] ==1, 'Success'] = "yes"
df_nba.loc[df_nba['Shot Made Flag'] ==0, 'Success'] = "no"
df =pd.crosstab(df_nba['Action Type'],df_nba['Success'],normalize = "index").reset_index()

df.loc[df["yes"] > 0,'CLASSE'] = "1"
df.loc[df["yes"] > 0.10,'CLASSE']="2"
df.loc[df["yes"] > 0.20,'CLASSE']="3"
df.loc[df["yes"] > 0.30,'CLASSE']="4"
df.loc[df["yes"] > 0.40,'CLASSE']="5"
df.loc[df["yes"] > 0.50,'CLASSE'] = "6"
df.loc[df["yes"] > 0.60,'CLASSE'] = "7"
df.loc[df["yes"] > 0.70 ,'CLASSE'] = "8"
df.loc[df["yes"] > 0.80 ,'CLASSE'] = "9"
df.loc[df["yes"] > 0.90 ,'CLASSE'] = "10"
df_nba=df_nba.merge(df,how='left')

#VAriables categorielles
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Basic']))
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Area']))
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Range']))
df_nba=df_nba.join(pd.get_dummies(df_nba['position']))

#1 joueur
names = ['Allen Iverson']   
df_nba_player=df_nba[df_nba['Player Name'].isin(names)]

to_drop = ['Game ID', 'Game Event ID', 'Player ID', 'Player Name',
           'Team ID', 'Team Name','Period', 'Minutes Remaining', 'Seconds Remaining', 
           'Action Type', 'Shot Type', 'Shot Zone Basic','Shot Zone Area', 'Shot Zone Range',
           'Shot Made Flag', 'Game Date', 'Home Team','Shot Distance','X Location','Y Location',  'Away Team', 'Season Type',
           'name', 'year_start', 'year_end','height','weight','birth_date',  'college', 'Success','no', 'yes','position']
data = df_nba_player.drop(to_drop,axis=1)
target=df_nba_player['Shot Made Flag']



# 1. as sidebar menu
with st.sidebar:
    selected = option_menu("Menu", ["Introduction", "Dataset","Analyse Descriptive", "Creation des Variables","Modelisation","Données Manquantes", "Exploration","Conclusion"], 
        icons=['house','123','image','keyboard','graph-up', 'gear','alarm','check'], menu_icon="cast", default_index=0)

#Page Introduction 

if selected=="Introduction":
  header=st.container()
  with header:
      st.subheader('*Introduction*')
      st.write("L’immense popularité des sports d’équipe aux Etats Unis et du Basket Ball par exemple, ont favorisé le développement de la collection de données sur ces sports.  Nous disposons donc de données extrêmement riches, collectées sur tous les aspects du sport, sur les joueurs, les matchs de Basket Ball, les tirs marqués ou non, et toutes les circonstances particulières dans lesquelles les tirs ont été effectués.") 
      st.write("En considérant les 20 meilleurs joueurs NBA du 21ème siècle selon la chaine de sport ESPN, nous allons analyser les tirs effectués par ces 20 joueurs, et développer des modèles prédictifs de Machine Learning sur ces tirs..")  

#Page Dataset

if selected=="Dataset":

  st.write('Pour notre étude, nous nous sommes essentiellement basés pour nos recherches sur le dataset des tirs NBA entre 1997 et 2020 (nba-shot-locations) ainsi que sur le dataset nommé players_data. Ces 2 datasets peuvent être téléchargés via le site Kaggle.')
  
  headerdataset1=st.container()
  headerdataset2=st.container()
  
  with headerdataset1:
      st.subheader('*1 )DATASET NBA SHOTS LOCATIONS*')
      st.write("Ce dataset recense tous les tirs des joueurs de la NBA de 1997 à 2000.") 
      st.write("Seuls les tirs à 2 points et 3 points sont mentionnés. On n’étudiera donc pas la précision des joueurs aux lancers francs.") 
      st.write("La variable **Shot Made Flag*** sera notre variable cible qui déterminera la réussite ou l’échec d’un tir.")
      st.write(df_nba_complete.head())
      st.write("")
      st.write("**On va restreindre par la suite ce dataset à 20 joueurs pour la modélisation.**")  
  with headerdataset2:
      st.subheader('*2) DATASET PLAYER DATA*')
      st.write("Ce dataset indique pour chaque joueur son nom, sa date de naissance, son poids, sa taille, son collège d’étude, son début et sa fin de carrière ainsi que sa position sur le terrain. ") 
      st.write(df_player.head())  

#Page Analyse Descriptive

if selected=="Analyse Descriptive":
      st.write("blabla")

#Page Analyse Descriptive

if selected=="Creation des Variables":
     
  headervariable1=st.container()
  headervariable2=st.container()
  headervariable3=st.container() 
  headervariable4=st.container()    
  headervariable5=st.container() 

  with headervariable1:   
      st.subheader('*1) Shot TIME*') 
      st.write("On a démontré que le pourcentage de réussite baissait en fonction des périodes. Cela se confirme pour nos 20 joueurs.")
      st.image("pourcentage de réussite.PNG",width=400)
      st.write("On a donc décidé d’enrichir notre data set avec la variable Shot_Time qui permet de savoir quand le shot a été effectué en tenant compte de la période, et du temps restant.")
           
      Periode=st.selectbox('**Choisissez la période**: ', options=[1,2,3,4,5,6,7])     
      Minutes=st.slider('**Choisissez les minutes restantes** : ', min_value=0,max_value=11) 
      Seconde=st.slider('**Choisissez les secondes restantes** : ', min_value=0,max_value=59) 
    
      if Periode <5:
          Shot_Time=(720-(60*Minutes+Seconde)+(Periode-1)*12*60)
      else:
          Shot_Time=(300-(60*Minutes+Seconde)+((Periode-5)*5+48)*60)

      col1, col2, col3 = st.columns([1,1,1])

      with col1:
        
          st.subheader("**Le shot a été effectué à la**") 
      with col2:
          st.subheader(Shot_Time)
      with col3:  
          st.subheader("**seconde du match**") 
      st.write("")
      st.write("")  

  with headervariable2:   
      st.subheader('*2) GROUPEMENT DES ACTIONS*') 
      st.write("L’analyse descriptive du dataframe principale nous a montré qu’il y avait 70 types d’actions")
      st.write("Ce nombre d’actions est beaucoup trop important .")  
      st.write("On a cherché une méthode de regroupement empirique. ")
      st.write("L’idée a été de calculer le pourcentage de réussite de ces actions à l’aide d’un crosstab puis de créer 10 classes en fonction de ces pourcentages. Dans la classe 10, on met toutes les actions qui ont un pourcentage de réussite supérieur à 90, puis dans la classe 9 celles qui ont un pourcentage de réussite compris entre 80 et 90. ")
      st.write("**On est donc passé de 70 actions à 10 classes**")
      st.write("")
      st.write("") 
  with headervariable3:   
      st.subheader('*3)	POSTE DES JOUEURS*') 
      st.write("Nous avons aussi ajouté au   data set principal le poste des joueurs. Pour cela nous avons lié notre data frame au data frame « player_data.csv » qui contient le poste des joueurs.")    
      st.write("")
      st.write("") 
  with headervariable4: 
      st.subheader('*4) VARIABLES CATEGORIELLES*') 
      st.write("Afin d’exploiter les variables catégorielles nous avons utilisé la méthode panda.get.dummies de python.")  
      st.write("Cette transformation a été appliquée au variables suivantes : ")
      st.write("*-Shot Zone Basic*" )  
      st.write("*-Shot Zone Area*" ) 
      st.write("*-Shot Zone Range*" ) 
      st.write("*-position*" )
      st.write("")
      st.write("")   
  with headervariable5: 
      st.subheader('*5) DATAFRAME FINAL*') 
      st.write("Nous avons évidemment enlevé toutes les colonnes id, toutes les colonnes catégorielles que nous avions transformées avec la méthode get.dummies. .")  
      st.write("Nous n’avons pas conservé  des colonnes que nous jugions inutiles.  Par exemple, nous avons enlevé le collège du joueur qui n’influence pas son taux de réussite.")
      st.write( "Ci-dessous un exemple des deux dataframe qui serviront  ")


#Page Exploration

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Titre
st.title("Le coefficient de Gini appliqué au basket ball")

#En-tête
st.header("La courbe de Lorenz")

#Ecrire texte
st.write("La courbe de Lorenz est une représentation graphique de la distribution d'une variable (revenu, patrimoine, etc.) au sein d'une population. Elle permet de calculer le coefficient de Gini, qui mesure l'inégalité de cette variable dans une population donnée. Dans notre cas, le but est de vérifier si la densité des joueurs au sein d'une équipe est homogène ou hétérogène.")

# Analyse de deux équipes

#En-tête
st.header("Deux équipes analysées: Les San Antonio Spurs et les Houston Rockets")

#Ecrire texte
st.write("Les San Antonio Spurs tendent à être plus collectif, se reposant sur un trio de joueurs performants en terme de moyenne de nombre de points tandis que les Houston Rockets est une équipe qui se repose plus sur les compétences d'un seul joueur")



#Moyenne du nombre de points de l'équipe des San Antonio Spurs de 2000 à 2010

#En-tête
st.header("Moyenne du nombre de points de l'équipe des San Antonio Spurs de 2000 à 2010")

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2000/2001")


df1 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


df1.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Shawnelle Scott','PTS':1.6,'Cpl':1})
df1.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Ira Newble','PTS':2,'Cpl':2})
df1.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Jaren Jackson','PTS':2.4,'Cpl':3})
df1.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Derrick Dial','PTS':2.6,'Cpl':4})
df1.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Steve Kerr','PTS':3.3,'Cpl':5})
df1.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Samaki Walker','PTS':5.3,'Cpl':6})
df1.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Danny Ferry','PTS':5.6,'Cpl':7})
df1.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Avery Johnson','PTS':5.6,'Cpl':8})
df1.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Terry Porter','PTS':7.2,'Cpl':9})
df1.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':7.7,'Cpl':10})
df1.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Sean Elliott','PTS':7.9,'Cpl':11})
df1.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Antonio Daniels','PTS':9.4,'Cpl':12})
df1.loc[13] = pd.Series({'Joueur San Antonio Spurs':'David Robinson','PTS':14.4,'Cpl':13})
df1.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Derek Anderson','PTS':15.5,'Cpl':14})
df1.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':22.2,'Cpl':15})

df1["CPTS"] = np.cumsum(df1["PTS"])

df1["Cpl%"] = df1["Cpl"].apply(lambda x : ((x/df1.loc[len(df1),"Cpl"])*100))

df1["CPTS%"] = df1["CPTS"].apply(lambda x : (x/df1.loc[len(df1),"CPTS"])*100)

if st.checkbox('Show dataframe 2000/2001'):
    st.write(df1)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2000/2001")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Saison 2000/2001

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.6,2,2.4,2.6,3.3,5.3,5.6,5.6,7.2,7.7,7.9,9.4,14.4,15.5,22.2])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2000/2001'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2000/2001'):
    st.write(fig)

#Saison 2001/2002

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2001/2002")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df20 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


df20.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Cherokee Parks','PTS':1.5,'Cpl':1})
df20.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Mark Bryant','PTS':1.9,'Cpl':2})
df20.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Amal McCaskill','PTS':1.9,'Cpl':3})
df20.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Jason Hart','PTS':2.6,'Cpl':4})
df20.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Stephen Jackson','PTS':3.9,'Cpl':5})
df20.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Danny Ferry','PTS':4.6,'Cpl':6})
df20.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Terry Porter','PTS':5.5,'Cpl':7})
df20.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':7,'Cpl':8})
df20.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Charles Smith','PTS':7.4,'Cpl':9})
df20.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':9.2,'Cpl':10})
df20.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Antonio Daniels','PTS':9.2,'Cpl':11})
df20.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':9.4,'Cpl':12})
df20.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Steve Smith','PTS':11.6,'Cpl':13})
df20.loc[14] = pd.Series({'Joueur San Antonio Spurs':'David Robinson','PTS':12.2,'Cpl':14})
df20.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':25.5,'Cpl':15})

df20["CPTS"] = np.cumsum(df20["PTS"])

df20["Cpl%"] = df20["Cpl"].apply(lambda x : ((x/df20.loc[len(df20),"Cpl"])*100))

df20["CPTS%"] = df20["CPTS"].apply(lambda x : (x/df20.loc[len(df20),"CPTS"])*100)

if st.checkbox('Show dataframe 2001/2002'):
    st.write(df20)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2001/2002")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.5,1.9,1.9,2.6,3.9,4.6,5.5,7,7.4,9.2,9.2,9.4,11.6,12.2,25.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2001/2002'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2001/2002'):
    st.write(fig)

#Saison 2002/2003

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2002/2003")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df21 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

df21.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Bateer Mengke','PTS':0.8,'Cpl':1})
df21.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Anthony Goldwire','PTS':1.2,'Cpl':2})
df21.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Devin Brown','PTS':1.7,'Cpl':3})
df21.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Danny Ferry','PTS':1.9,'Cpl':4})
df21.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Steve Kerr','PTS':4,'Cpl':5})
df21.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Kevin Willis','PTS':4.2,'Cpl':6})
df21.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Speedy Claxton','PTS':5.8,'Cpl':7})
df21.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Steve Smith','PTS':6.8,'Cpl':8})
df21.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':7.1,'Cpl':9})
df21.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':7.6,'Cpl':10})
df21.loc[11] = pd.Series({'Joueur San Antonio Spurs':'David Robinson','PTS':8.5,'Cpl':11})
df21.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':10.4,'Cpl':12})
df21.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Stephen Jackson','PTS':11.8,'Cpl':13})
df21.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':15.5,'Cpl':14})
df21.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':23.3,'Cpl':15})

df21["CPTS"] = np.cumsum(df21["PTS"])

df21["Cpl%"] = df21["Cpl"].apply(lambda x : ((x/df21.loc[len(df21),"Cpl"])*100))

df21["CPTS%"] = df21["CPTS"].apply(lambda x : (x/df21.loc[len(df21),"CPTS"])*100)

if st.checkbox('Show dataframe 2002/2003'):
    st.write(df21)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2002/2003")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0.8,1.2,1.7,1.9,4,4.2,5.8,6.8,7.1,7.6,8.5,10.4,11.8,15.5,23.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2002/2003'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2002/2003'):
    st.write(fig)


#Saison 2003/2004

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2003/2004")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df22 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df22.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Alex Garcia','PTS':1.5,'Cpl':1})
df22.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Matt Carroll','PTS':2,'Cpl':2})
df22.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Charlie Ward','PTS':3.3,'Cpl':3})
df22.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Jason Hart','PTS':3.3,'Cpl':4})
df22.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Kevin Willis','PTS':3.4,'Cpl':5})
df22.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Shane Heal','PTS':3.7,'Cpl':6})
df22.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Devin Brown','PTS':4,'Cpl':7})
df22.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Anthony Carter','PTS':4.4,'Cpl':8})
df22.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Robert Horry','PTS':4.8,'Cpl':9})
df22.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Ron Mercer','PTS':5,'Cpl':10})
df22.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':6.9,'Cpl':11})
df22.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':7.9,'Cpl':12})
df22.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Radoslav Nesterovic','PTS':8.7,'Cpl':13})
df22.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Hidayet Turkoglu','PTS':9.2,'Cpl':14})
df22.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':12.8,'Cpl':15})
df22.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':14.7,'Cpl':16})
df22.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':22.3,'Cpl':17})

df22["CPTS"] = np.cumsum(df22["PTS"])

df22["Cpl%"] = df22["Cpl"].apply(lambda x : ((x/df22.loc[len(df22),"Cpl"])*100))

df22["CPTS%"] = df22["CPTS"].apply(lambda x : (x/df22.loc[len(df22),"CPTS"])*100)

if st.checkbox('Show dataframe 2003/2004'):
    st.write(df22)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2003/2004")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.5,2,3.3,3.3,3.4,3.7,4,4.4,4.8,5,6.9,7.9,8.7,9.2,12.8,14.7,22.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2003/2004'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2003/2004'):
    st.write(fig)


#Saison 2004/2005

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2004/2005")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt    

df23 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df23.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Linton Johnson','PTS':0,'Cpl':1})
df23.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Mike Wilks','PTS':1.7,'Cpl':2})
df23.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Tony Massenburg','PTS':3.2,'Cpl':3})
df23.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Sean Marks','PTS':3.3,'Cpl':4})
df23.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Dion Glover','PTS':3.6,'Cpl':5})
df23.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Radoslav Nesterovic','PTS':5.9,'Cpl':6})
df23.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Beno Udrih','PTS':5.9,'Cpl':7})
df23.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Robert Horry','PTS':6,'Cpl':8})
df23.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Nazr Mohammed','PTS':6.2,'Cpl':9})
df23.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':6.3,'Cpl':10})
df23.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Devin Brown','PTS':7.4,'Cpl':11})
df23.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Brent Barry','PTS':7.4,'Cpl':12})
df23.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':8.2,'Cpl':13})
df23.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Glenn Robinson','PTS':10,'Cpl':14})
df23.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':16,'Cpl':15})
df23.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':16.6,'Cpl':16})
df23.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':20.3,'Cpl':17})

df23["CPTS"] = np.cumsum(df23["PTS"])

df23["Cpl%"] = df23["Cpl"].apply(lambda x : ((x/df23.loc[len(df23),"Cpl"])*100))

df23["CPTS%"] = df23["CPTS"].apply(lambda x : (x/df23.loc[len(df23),"CPTS"])*100)

if st.checkbox('Show dataframe 2004/2005'):
    st.write(df23)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2004/2005")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))    

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,1.7,3.2,3.3,3.6,5.9,5.9,6,6.2,6.3,7.4,7.4,8.2,10,16,16.6,20.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2004/2005'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2004/2005'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2005/2006")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df24 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

df24.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Alex Scales','PTS':0,'Cpl':1})
df24.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Fabricio Oberto','PTS':1.7,'Cpl':2})
df24.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Melvin Sanders','PTS':2.6,'Cpl':3})
df24.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Sean Marks','PTS':3.2,'Cpl':4})
df24.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Radoslav Nesterovic','PTS':4.5,'Cpl':5})
df24.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Beno Udrih','PTS':5.1,'Cpl':6})
df24.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Robert Horry','PTS':5.1,'Cpl':7})
df24.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Nick Van Exel','PTS':5.5,'Cpl':8})
df24.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Brent Barry','PTS':5.8,'Cpl':9})
df24.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Nazr Mohammed','PTS':6.2,'Cpl':10})
df24.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':7.5,'Cpl':11})
df24.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Michael Finley','PTS':10.1,'Cpl':12})
df24.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':15.1,'Cpl':13})
df24.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':18.6,'Cpl':14})
df24.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':18.9,'Cpl':15})


df24["CPTS"] = np.cumsum(df24["PTS"])

df24["Cpl%"] = df24["Cpl"].apply(lambda x : ((x/df24.loc[len(df24),"Cpl"])*100))

df24["CPTS%"] = df24["CPTS"].apply(lambda x : (x/df24.loc[len(df24),"CPTS"])*100)

if st.checkbox('Show dataframe 2005/2006'):
    st.write(df24)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2005/2006")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,1.7,2.6,3.2,4.5,5.1,5.1,5.5,5.8,6.2,7.5,10.1,15.1,18.6])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2005/2006'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2005/2006'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2006/2007")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df25 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

df25.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Eric Williams','PTS':2.6,'Cpl':1})
df25.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Jacque Vaughn','PTS':3,'Cpl':2})
df25.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Melvin Ely','PTS':3.2,'Cpl':3})
df25.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Jackie Butler','PTS':3.7,'Cpl':4})
df25.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Robert Horry','PTS':3.9,'Cpl':5})
df25.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Fabricio Oberto','PTS':4.4,'Cpl':6})
df25.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Beno Udrih','PTS':4.7,'Cpl':7})
df25.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':4.9,'Cpl':8})
df25.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Francisco Elson','PTS':5,'Cpl':9})
df25.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':6.2,'Cpl':10})
df25.loc[11] = pd.Series({'Joueur San Antonio Spurs':'James White','PTS':8.3,'Cpl':11})
df25.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Brent Barry','PTS':8.5,'Cpl':12})
df25.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Michael Finley','PTS':9,'Cpl':13})
df25.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':16.5,'Cpl':14})
df25.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':18.6,'Cpl':15})
df25.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':20,'Cpl':16})


df25["CPTS"] = np.cumsum(df25["PTS"])

df25["Cpl%"] = df25["Cpl"].apply(lambda x : ((x/df25.loc[len(df25),"Cpl"])*100))

df25["CPTS%"] = df25["CPTS"].apply(lambda x : (x/df25.loc[len(df25),"CPTS"])*100)

if st.checkbox('Show dataframe 2006/2007'):
    st.write(df25)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2006/2007")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([2.6,3,3.2,3.7,3.9,4.4,4.7,4.9,5,6.2,8.3,8.5,9,16.5,18.6,20])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2006/2007'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2006/2007'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2007/2008")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df26 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

df26.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Marcus Williams','PTS':0,'Cpl':1})
df26.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Bobby Jones','PTS':0.7,'Cpl':2})
df26.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Keith Langford','PTS':1,'Cpl':3})
df26.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Jeremy Richardson','PTS':2,'Cpl':4})
df26.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Robert Horry','PTS':2.5,'Cpl':5})
df26.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Darius Washington','PTS':2.9,'Cpl':6})
df26.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Dermarr Johnson','PTS':3.4,'Cpl':7})
df26.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Damon Stoudamire','PTS':3.4,'Cpl':8})
df26.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Ian Mahinmi','PTS':3.5,'Cpl':9})
df26.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Francisco Elson','PTS':3.5,'Cpl':10})
df26.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Jacque Vaughn','PTS':4.1,'Cpl':11})
df26.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Kurt Thomas','PTS':4.5,'Cpl':12})
df26.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':4.8,'Cpl':13})
df26.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Fabricio Oberto','PTS':4.8,'Cpl':14})
df26.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Ime Udoka','PTS':5.8,'Cpl':15})
df26.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':6,'Cpl':16})
df26.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Brent Barry','PTS':7.1,'Cpl':17})
df26.loc[18] = pd.Series({'Joueur San Antonio Spurs':'Michael Finley','PTS':10.1,'Cpl':18})
df26.loc[19] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':18.8,'Cpl':19})
df26.loc[20] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':19.3,'Cpl':20})
df26.loc[21] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':19.5,'Cpl':21})

df26["CPTS"] = np.cumsum(df26["PTS"])

df26["Cpl%"] = df26["Cpl"].apply(lambda x : ((x/df26.loc[len(df26),"Cpl"])*100))

df26["CPTS%"] = df26["CPTS"].apply(lambda x : (x/df26.loc[len(df26),"CPTS"])*100)

if st.checkbox('Show dataframe 2007/2008'):
    st.write(df26)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2007/2008")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,0.7,1,2,2.5,2.9,3.4,3.4,3.5,3.5,4.1,4.5,4.8,4.8,5.8,6,7.1,10.1,18.8,19.3,19.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2007/2008'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2007/2008'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2008/2009")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df27 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

df27.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Austin Croshere','PTS':1.3,'Cpl':1})
df27.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Jacque Vaughn','PTS':2.2,'Cpl':2})
df27.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Blake Ahearn','PTS':2.7,'Cpl':3})
df27.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Bruce Bowen','PTS':2.7,'Cpl':4})
df27.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Fabricio Oberto','PTS':2.7,'Cpl':5})
df27.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Anthony Tolliver','PTS':2.7,'Cpl':6})
df27.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Malik Hairston','PTS':3.6,'Cpl':7})
df27.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Marcus Williams','PTS':4,'Cpl':8})
df27.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Desmon Farmer','PTS':4.3,'Cpl':9})
df27.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Kurt Thomas','PTS':4.3,'Cpl':10})
df27.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Ime Udoka','PTS':4.4,'Cpl':15})
df27.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Pops Mensah-Bonsu','PTS':5,'Cpl':16})
df27.loc[13] = pd.Series({'Joueur San Antonio Spurs':'George Hill','PTS':5.8,'Cpl':17})
df27.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':8.2,'Cpl':18})
df27.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Michael Finley','PTS':9.7,'Cpl':19})
df27.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Drew Gooden','PTS':9.8,'Cpl':20})
df27.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Roger jr. Mason','PTS':11.8,'Cpl':21})
df27.loc[18] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':15.5,'Cpl':22})
df27.loc[19] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':19.3,'Cpl':23})
df27.loc[20] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':22,'Cpl':24})

df27["CPTS"] = np.cumsum(df27["PTS"])

df27["Cpl%"] = df27["Cpl"].apply(lambda x : ((x/df27.loc[len(df27),"Cpl"])*100))

df27["CPTS%"] = df27["CPTS"].apply(lambda x : (x/df27.loc[len(df27),"CPTS"])*100)

if st.checkbox('Show dataframe 2008/2009'):
    st.write(df27)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2008/2009")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.3,2.2,2.7,2.7,2.7,2.7,3.6,4,4.3,4.3,4.4,5,5.8,8.2,9.7,9.8,11.8,15.5,19.3,22])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2008/2009'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2008/2009'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2009/2010")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df28 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df28.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Theo Ratliff','PTS':1.6,'Cpl':1})
df28.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Malik Hairston','PTS':2.3,'Cpl':2})
df28.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Cedric Jackson','PTS':2.3,'Cpl':3})
df28.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Marcus Haislip','PTS':2.5,'Cpl':4})
df28.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Michael Finley','PTS':3.7,'Cpl':5})
df28.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Ian Mahinmi','PTS':4,'Cpl':6})
df28.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Keith Bogans','PTS':4.4,'Cpl':7})
df28.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Antonio McDyess','PTS':5.8,'Cpl':8})
df28.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Garrett Temple','PTS':6.2,'Cpl':9})
df28.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Roger jr. Mason','PTS':6.3,'Cpl':10})
df28.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':7.1,'Cpl':15})
df28.loc[12] = pd.Series({'Joueur San Antonio Spurs':'DeJuan Blair','PTS':7.8,'Cpl':16})
df28.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Richard Jefferson','PTS':12.3,'Cpl':17})
df28.loc[14] = pd.Series({'Joueur San Antonio Spurs':'George Hill','PTS':12.3,'Cpl':18})
df28.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':16,'Cpl':19})
df28.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':16.5,'Cpl':20})
df28.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':17.9,'Cpl':21})

df28["CPTS"] = np.cumsum(df28["PTS"])

df28["Cpl%"] = df28["Cpl"].apply(lambda x : ((x/df28.loc[len(df28),"Cpl"])*100))

df28["CPTS%"] = df28["CPTS"].apply(lambda x : (x/df28.loc[len(df28),"CPTS"])*100)

if st.checkbox('Show dataframe 2009/2010'):
    st.write(df28)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2009/2010")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))  

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.6,2.3,2.3,2.5,3.7,4,4.4,5.8,6.2,6.3,7.1,7.8,12.3,12.3,16,16.5,17.9])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2009/2010'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2009/2010'):
    st.write(fig)

#En-tête
st.header("Moyenne du nombre de points de l'équipe des San Antonio Spurs de 2010 à 2018")


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2010/2011")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df29 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

df29.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Bobby Simmons','PTS':0,'Cpl':1})
df29.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Alonzo Gee','PTS':0.7,'Cpl':2})
df29.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Garrett Temple','PTS':0.7,'Cpl':3})
df29.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Ime Udoka','PTS':0.8,'Cpl':4})
df29.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Larry Owens','PTS':1.3,'Cpl':5})
df29.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Othyus Jeffers','PTS':2,'Cpl':6})
df29.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Chris Quinn','PTS':2.1,'Cpl':7})
df29.loc[8] = pd.Series({'Joueur San Antonio Spurs':'James Anderson','PTS':3.9,'Cpl':8})
df29.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Steve Novak','PTS':4,'Cpl':9})
df29.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Tiago Splitter','PTS':4.7,'Cpl':10})
df29.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Antonio McDyess','PTS':5.3,'Cpl':11})
df29.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':5.9,'Cpl':12})
df29.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':7.3,'Cpl':13})
df29.loc[14] = pd.Series({'Joueur San Antonio Spurs':'DeJuan Blair','PTS':8.3,'Cpl':14})
df29.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Gary Neal','PTS':9.8,'Cpl':15})
df29.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Richard Jefferson','PTS':11,'Cpl':16})
df29.loc[17] = pd.Series({'Joueur San Antonio Spurs':'George Hill','PTS':11.6,'Cpl':17})
df29.loc[18] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':13.4,'Cpl':18})
df29.loc[19] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':17.4,'Cpl':19})
df29.loc[20] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':17.5,'Cpl':20})


df29["CPTS"] = np.cumsum(df29["PTS"])

df29["Cpl%"] = df29["Cpl"].apply(lambda x : ((x/df29.loc[len(df29),"Cpl"])*100))

df29["CPTS%"] = df29["CPTS"].apply(lambda x : (x/df29.loc[len(df29),"CPTS"])*100)

if st.checkbox('Show dataframe 2010/2011'):
    st.write(df29)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2010/2011")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))  

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,0.7,0.7,0.8,1.3,2,2.1,3.9,4,4.7,5.3,5.9,7.3,8.3,9.8,11,11.6,13.4,17.4,17.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2010/2011'):
  st.write(gini(arr))


lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2010/2011'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2011/2012")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df30 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

df30.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Malcolm Thomas','PTS':0.3,'Cpl':1})
df30.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Justin Dentmon','PTS':2,'Cpl':2})
df30.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Ike Diogu','PTS':2,'Cpl':3})
df30.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Cory Joseph','PTS':2.2,'Cpl':4})
df30.loc[5] = pd.Series({'Joueur San Antonio Spurs':'T.J. Ford','PTS':3.6,'Cpl':5})
df30.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Eric Dawson','PTS':3.8,'Cpl':6})
df30.loc[7] = pd.Series({'Joueur San Antonio Spurs':'James Anderson','PTS':3.8,'Cpl':7})
df30.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Boris Diaw','PTS':4.7,'Cpl':8})
df30.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Derrick Byars','PTS':5,'Cpl':9})
df30.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':6.7,'Cpl':10})
df30.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Kawhi Leonard','PTS':7.9,'Cpl':11})
df30.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Stephen Jackson','PTS':8.9,'Cpl':12})
df30.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':9.1,'Cpl':13})
df30.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Richard Jefferson','PTS':9.2,'Cpl':14})
df30.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tiago Splitter','PTS':9.3,'Cpl':15})
df30.loc[16] = pd.Series({'Joueur San Antonio Spurs':'DeJuan Blair','PTS':9.5,'Cpl':16})
df30.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Gary Neal','PTS':9.9,'Cpl':17})
df30.loc[18] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':10.3,'Cpl':18})
df30.loc[19] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':12.9,'Cpl':19})
df30.loc[20] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':15.4,'Cpl':20})
df30.loc[21] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':18.3,'Cpl':21})

df30["CPTS"] = np.cumsum(df30["PTS"])

df30["Cpl%"] = df30["Cpl"].apply(lambda x : ((x/df30.loc[len(df30),"Cpl"])*100))

df30["CPTS%"] = df30["CPTS"].apply(lambda x : (x/df30.loc[len(df30),"CPTS"])*100)

if st.checkbox('Show dataframe 2011/2012'):
    st.write(df30)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2011/2012")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0.3,2,2,2.2,3.6,3.8,3.8,4.7,5,6.7,7.9,8.9,9.1,9.2,9.3,9.5,9.9,10.3,12.9,15.4,18.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2011/2012'):
  st.write(gini(arr))


lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2011/2012'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2012/2013")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df31 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

df31.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Aron Baynes','PTS':3.1,'Cpl':1})
df31.loc[2] = pd.Series({'Joueur San Antonio Spurs':'James Anderson','PTS':3.8,'Cpl':2})
df31.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Nando De Colo','PTS':3.9,'Cpl':3})
df31.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':4.2,'Cpl':4})
df31.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Cory Joseph','PTS':5,'Cpl':5})
df31.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':5.3,'Cpl':6})
df31.loc[7] = pd.Series({'Joueur San Antonio Spurs':'DeJuan Blair','PTS':5.5,'Cpl':7})
df31.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Boris Diaw','PTS':5.8,'Cpl':8})
df31.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Stephen Jackson','PTS':6.2,'Cpl':9})
df31.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Gary Neal','PTS':9.5,'Cpl':10})
df31.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Tiago Splitter','PTS':10.3,'Cpl':11})
df31.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':10.5,'Cpl':12})
df31.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':11.8,'Cpl':13})
df31.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Kawhi Leonard','PTS':11.9,'Cpl':14})
df31.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':17.8,'Cpl':15})
df31.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':20.3,'Cpl':16})

df31["CPTS"] = np.cumsum(df31["PTS"])

df31["Cpl%"] = df31["Cpl"].apply(lambda x : ((x/df31.loc[len(df31),"Cpl"])*100))

df31["CPTS%"] = df31["CPTS"].apply(lambda x : (x/df31.loc[len(df31),"CPTS"])*100)

if st.checkbox('Show dataframe 2012/2013'):
    st.write(df31)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2012/2013")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([3.1,3.8,3.9,4.2,5,5.3,5.5,5.8,6.2,9.5,10.3,10.5,11.8,11.9,17.8,20.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2012/2013'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2012/2013'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2013/2014")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df32 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

df32.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Malcolm Thomas','PTS':1,'Cpl':1})
df32.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Damion James','PTS':1.5,'Cpl':2})
df32.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Othyus Jeffers','PTS':1.8,'Cpl':3})
df32.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Shannon Brown','PTS':2.6,'Cpl':4})
df32.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Aron Baynes','PTS':3.1,'Cpl':5})
df32.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Jeff Ayres','PTS':3.3,'Cpl':6})
df32.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':3.3,'Cpl':7})
df32.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Austin Daye','PTS':4.1,'Cpl':8})
df32.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Nando De Colo','PTS':4.3,'Cpl':9})
df32.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Cory Joseph','PTS':5.4,'Cpl':10})
df32.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Tiago Splitter','PTS':8.2,'Cpl':11})
df32.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Boris Diaw ','PTS':9.1,'Cpl':12})
df32.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':9.1,'Cpl':13})
df32.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':10.2,'Cpl':14})
df32.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Marco Belinelli','PTS':11.4,'Cpl':15})
df32.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':12.3,'Cpl':16})
df32.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Kahwi Leonard','PTS':12.8,'Cpl':17})
df32.loc[18] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':15.1,'Cpl':18})
df32.loc[19] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':16.7,'Cpl':19})

df32["CPTS"] = np.cumsum(df32["PTS"])

df32["Cpl%"] = df32["Cpl"].apply(lambda x : ((x/df32.loc[len(df32),"Cpl"])*100))

df32["CPTS%"] = df32["CPTS"].apply(lambda x : (x/df32.loc[len(df32),"CPTS"])*100)

if st.checkbox('Show dataframe 2013/2014'):
    st.write(df32)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2013/2014")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1,1.5,1.8,2.6,3.1,3.3,3.3,4.1,4.3,5.4,8.2,9.1,9.1,10.2,11.4,12.3,12.8,15.1,16.7])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2013/2014'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2013/2014'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2014/2015")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df33 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df33.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Reggie Williams','PTS':1.9,'Cpl':1})
df33.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Jamychal Green','PTS':2,'Cpl':2})
df33.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Kyle Anderson','PTS':2.2,'Cpl':3})
df33.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Jeff Ayres','PTS':2.9,'Cpl':4})
df33.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':3.8,'Cpl':5})
df33.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Austin Daye','PTS':4.3,'Cpl':6})
df33.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Aron Baynes','PTS':6.6,'Cpl':7})
df33.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Cory Joseph','PTS':6.9,'Cpl':8})
df33.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':6.9,'Cpl':9})
df33.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Tiago Splitter','PTS':8.2,'Cpl':10})
df33.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Boris Diaw','PTS':8.7,'Cpl':11})
df33.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Marco Belinelli ','PTS':9.2,'Cpl':12})
df33.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':10.5,'Cpl':13})
df33.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':11.7,'Cpl':14})
df33.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':13.9,'Cpl':15})
df33.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':14.4,'Cpl':16})
df33.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Kahwi Leonard','PTS':16.5,'Cpl':17})

df33["CPTS"] = np.cumsum(df33["PTS"])

df33["Cpl%"] = df33["Cpl"].apply(lambda x : ((x/df33.loc[len(df33),"Cpl"])*100))

df33["CPTS%"] = df33["CPTS"].apply(lambda x : (x/df33.loc[len(df33),"CPTS"])*100)

if st.checkbox('Show dataframe 2014/2015'):
    st.write(df33)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2014/2015")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.9,2,2.2,2.9,3.8,4.3,6.6,6.9,6.9,8.2,8.7,9.2,10.5,11.7,13.9,14.4,16.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2014/2015'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2014/2015'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2015/2016")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df1 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df1.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Ray McCallum','PTS':2.3,'Cpl':1})
df1.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Rasual Butler','PTS':2.7,'Cpl':2})
df1.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Matt Bonner','PTS':2.8,'Cpl':3})
df1.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Andre Miller','PTS':4.3,'Cpl':4})
df1.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Kyle Anderson','PTS':4.5,'Cpl':5})
df1.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Boban Marjanovic','PTS':5.5,'Cpl':6})
df1.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Jonathon Simmons','PTS':6,'Cpl':7})
df1.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Kevin Martin','PTS':6.2,'Cpl':8})
df1.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Boris Diaw','PTS':6.4,'Cpl':9})
df1.loc[10] = pd.Series({'Joueur San Antonio Spurs':'David West','PTS':7.1,'Cpl':10})
df1.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':7.2,'Cpl':11})
df1.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':8.5,'Cpl':12})
df1.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':8.6,'Cpl':13})
df1.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':9.6,'Cpl':14})
df1.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':11.9,'Cpl':15})
df1.loc[16] = pd.Series({'Joueur San Antonio Spurs':'LaMarcus Aldridger','PTS':18,'Cpl':16})
df1.loc[17] = pd.Series({'Joueur San Antonio Spurs':'Kahwi Leonard','PTS':21.2,'Cpl':17})

df1["CPTS"] = np.cumsum(df1["PTS"])

df1["Cpl%"] = df1["Cpl"].apply(lambda x : ((x/df1.loc[len(df1),"Cpl"])*100))

df1["CPTS%"] = df1["CPTS"].apply(lambda x : (x/df1.loc[len(df1),"CPTS"])*100)

if st.checkbox('Show dataframe 2015/2016'):
    st.write(df1)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2015/2016")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([2.3,2.7,2.8,4.3,4.5,5.5,6,6.2,6.4,7.1,7.2,8.5,8.6,9.6,11.9,18,21.2])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2015/2016'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2015/2016'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2016/2017")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df1 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

df1.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Joel Anthony','PTS':1.4,'Cpl':1})
df1.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Bryn Forbes','PTS':2.8,'Cpl':2})
df1.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Kyle Anderson','PTS':3.5,'Cpl':3})
df1.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Nicolas Laprovittola','PTS':3.7,'Cpl':4})
df1.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Dejounte Murray','PTS':3.8,'Cpl':5})
df1.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Davis Bertans','PTS':4.7,'Cpl':6})
df1.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Dewayne Dedmon','PTS':5.1,'Cpl':7})
df1.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Jonathon Simmons','PTS':6.1,'Cpl':8})
df1.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':7.4,'Cpl':9})
df1.loc[10] = pd.Series({'Joueur San Antonio Spurs':'David Lee','PTS':7.4,'Cpl':10})
df1.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':7.5,'Cpl':11})
df1.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':9.5,'Cpl':12})
df1.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':10.2,'Cpl':13})
df1.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Pau Gasol','PTS':12.4,'Cpl':14})
df1.loc[15] = pd.Series({'Joueur San Antonio Spurs':'LaMarcus Aldridge','PTS':17.2,'Cpl':15})
df1.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Kawhi Leonard','PTS':25.6,'Cpl':16})

df1["CPTS"] = np.cumsum(df1["PTS"])

df1["Cpl%"] = df1["Cpl"].apply(lambda x : ((x/df1.loc[len(df1),"Cpl"])*100))

df1["CPTS%"] = df1["CPTS"].apply(lambda x : (x/df1.loc[len(df1),"CPTS"])*100)

if st.checkbox('Show dataframe 2016/2017'):
    st.write(df1)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2016/2017")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.4,2.8,3.5,3.7,3.8,4.7,5.1,6.1,7.4,7.4,7.5,9.5,10.2,12.4,17.2,25.6])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2016/2017'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2016/2017'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2017/2018")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df1 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df1.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Matt Costello','PTS':1,'Cpl':1})
df1.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Darrun Hilliard','PTS':1.1,'Cpl':2})
df1.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Brandon Paul','PTS':2.5,'Cpl':3})
df1.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Derrick White','PTS':3.6,'Cpl':4})
df1.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Joffrey Lauvergne','PTS':4.1,'Cpl':5})
df1.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Davis Bertans','PTS':6,'Cpl':6})
df1.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Bryn Forbes','PTS':7,'Cpl':7})
df1.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Tony Parker','PTS':7.7,'Cpl':8})
df1.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Kyle Anderson','PTS':7.9,'Cpl':9})
df1.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Dejounte Murray','PTS':8.2,'Cpl':10})
df1.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Danny Green','PTS':8.6,'Cpl':11})
df1.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Manu Ginobili','PTS':8.9,'Cpl':12})
df1.loc[13] = pd.Series({'Joueur San Antonio Spurs':'Patty Mills','PTS':10,'Cpl':13})
df1.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Pau Gasol','PTS':10.1,'Cpl':14})
df1.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Rudy Gay','PTS':11.5,'Cpl':15})
df1.loc[16] = pd.Series({'Joueur San Antonio Spurs':'Kawhi Leonard','PTS':16.2,'Cpl':16})
df1.loc[17] = pd.Series({'Joueur San Antonio Spurs':'LaMarcus Aldridge','PTS':16.2,'Cpl':17})

df1["CPTS"] = np.cumsum(df1["PTS"])

df1["Cpl%"] = df1["Cpl"].apply(lambda x : ((x/df1.loc[len(df1),"Cpl"])*100))

df1["CPTS%"] = df1["CPTS"].apply(lambda x : (x/df1.loc[len(df1),"CPTS"])*100)

if st.checkbox('Show dataframe 2017/2018'):
    st.write(df1)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2017/2018")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#San antonio Spurs

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1,1.1,2.5,3.6,4.1,6,7,7.7,7.9,8.2,8.6,8.9,10,10.1,11.5,16.2,23.1])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini 2017/2018'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz 2017/2018'):
    st.write(fig)


#En-tête
st.header("Moyenne du nombre de points de l'équipe des Houston Rockets de 2000 à 2010")


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2000/2001")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Anthony Miller','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Sean Colson','PTS':1.5,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Dan Langhi','PTS':2.7,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Jason Collier','PTS':3.1,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Carlos Rogers','PTS':4.6,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Kelvin Cato','PTS':4.7,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Matt Bullard','PTS':5.8,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':6.6,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Kenny Thomas','PTS':7.1,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Walt Williams','PTS':8.3,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Shandon Anderson','PTS':8.7,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Hakeem Olajuwon','PTS':11.9,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Maurice Taylor','PTS':13,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Cuttino Mobley','PTS':19.5,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Steve Francis','PTS':19.9,'Cpl':15})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2000/2001'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2000/2001")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,1.5,2.7,3.1,4.6,4.7,5.8,6.6,7.1,8.3,8.7,11.9,13,19.5,19.9])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2000/2001'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2000/2001'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2001/2002")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Tierre Brown','PTS':3.1,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Dan Langhi','PTS':3.1,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Terence Morris','PTS':3.8,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Jason Collier','PTS':4.2,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Oscar Torres','PTS':6,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Kevin Willis','PTS':6.1,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Kelvin Cato','PTS':6.6,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':8.1,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Glen Rice','PTS':8.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Eddie Griffin','PTS':8.8,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Walt Williams','PTS':9.4,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Kenny Thomas','PTS':14.1,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Steve Francis','PTS':21.6,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Cuttino Mobley','PTS':21.7,'Cpl':14})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2001/2002'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2001/2002")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([3.1,3.1,3.8,4.2,6,6.1,6.6,8.1,8.6,8.8,9.4,14.1,21.6,21.7])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2001/2002'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2001/2002'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2002/2003")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Tito Maddox','PTS':1.2,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Bostjan Nachbar','PTS':2.1,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Juaquin Hawkins','PTS':2.3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Jason Collier','PTS':2.8,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Terence Morris','PTS':3.7,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':4.4,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Kelvin Cato','PTS':4.5,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Maurice Taylor','PTS':8.4,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Eddie Griffin','PTS':8.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Glen Rice','PTS':9,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'James Posey','PTS':9.3,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Kenny Thomas','PTS':9.8,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':13.5,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Cuttino Mobley','PTS':17.5,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Steve Francis','PTS':21,'Cpl':15})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2002/2003'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2002/2003")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.2,2.1,2.3,2.8,3.7,4.4,4.5,8.4,8.6,9,9.3,9.8,13.5,17.5,21])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2002/2003'):
  st.write(gini(arr))


lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2002/2003'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2003/2004")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Adrian Griffin','PTS':0.6,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Charles Oakley','PTS':1.3,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Alton Ford','PTS':1.7,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Mike Wilks','PTS':2,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Mark Jackson','PTS':2.5,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':2.7,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Bostjan Nachbar','PTS':3.1,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Torraye Braggs','PTS':3.1,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Scott Padgett','PTS':3.4,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Eric Piatkowski','PTS':4.1,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Clarence Weatherspoon','PTS':5.6,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Kelvin Cato','PTS':6.1,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Maurice Taylor','PTS':11.5,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Jim Jackson','PTS':12.9,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Cuttino Mobley','PTS':15.8,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Steve Francis','PTS':16.6,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':17.5,'Cpl':17})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2003/2004'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2003/2004")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([15.8,16.6,17.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2003/2004'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2003/2004'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2004/2005")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Brandin Knight','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Vin Baker','PTS':0.7,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Torraye Braggs','PTS':0.9,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':1.3,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Ryan Bowen','PTS':1.7,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Rod Strickland','PTS':1.8,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Andre Barrett','PTS':2.1,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Reece Gaines','PTS':2.6,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Bostjan Nachbar','PTS':3.1,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Clarence Weatherspoon','PTS':3.1,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Dikembe Mutombo','PTS':4,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Scott Padgett','PTS':4.2,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Charlie Ward','PTS':5.4,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Tyronn Lue','PTS':6,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Jon Barry','PTS':7,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Maurice Taylor','PTS':7.8,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Juwan Howard','PTS':9.6,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Bob Sura','PTS':10.3,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'David Wesley','PTS':10.9,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Mike James','PTS':12.4,'Cpl':20})
df3.loc[21] = pd.Series({'Joueur Houston Rockets':'Jim Jackson','PTS':13.3,'Cpl':21})
df3.loc[22] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':18.3,'Cpl':22})
df3.loc[23] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':25.7,'Cpl':23})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2004/2005'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2004/2005")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,0.7,0.9,1.3,1.7,1.8,2.1,2.6,3.1,3.1,4,4.2,5.4,6,7,7.8,9.6,10.3,10.9,12.4,13.3,18.3,25.7])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2004/2005'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2004/2005'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2005/2006")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Josh Davis','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Maciej Lampe','PTS':1,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Ryan Bowen','PTS':1.3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Rick Brunson','PTS':1.9,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Moochie Norris','PTS':2.2,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'John Lucas','PTS':2.3,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Dikembe Mutombo','PTS':2.6,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Stephen Graham','PTS':2.8,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Lonny Baxter','PTS':3.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':3.7,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Jon Barry','PTS':4.5,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Richard Frahm','PTS':5.3,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Keith Bogans','PTS':8.5,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Luther Head','PTS':8.8,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Stromile Swift','PTS':8.9,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'David Wesley','PTS':9.9,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Derek Anderson','PTS':10.8,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Juwan Howard','PTS':11.8,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Rafer Alston','PTS':12.1,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':22.3,'Cpl':20})
df3.loc[21] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':24.4,'Cpl':21})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2005/2006'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2005/2006")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,1,1.3,1.9,2.2,2.3,2.6,2.6,2.8,3.6,3.7,4.5,5.3,8.5,8.8,8.9,9.9,10.8,11.8,12.1,22.3,24.4])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2005/2006'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2005/2006'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2006/2007")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Steve Novak','PTS':1.5,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Scott Padgett','PTS':1.8,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Iakovos Tsakalidis','PTS':2.3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Vassilis Spanoulis','PTS':2.7,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Dikembe Mutombo','PTS':3.1,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'John Lucas','PTS':3.3,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Kirk Snyder','PTS':4.9,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':5.6,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Bonzi Wells','PTS':7.8,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Juwan Howard','PTS':9.7,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Shane Battier','PTS':10.1,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Luther Head','PTS':10.9,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Rafer Alston','PTS':13.2,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':24.6,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':25,'Cpl':15})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2006/2007'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2006/2007")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.5,1.8,2.3,2.7,3.1,3.3,4.9,5.6,7.8,9.7,10.1,10.9,13.2,24.6])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2006/2007'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2006/2007'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2007/2008")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Loren Woods','PTS':1,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Bobby Jones','PTS':1,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Justin Williams','PTS':3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':3,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Dikembe Mutombo','PTS':3,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Mike Harris','PTS':3.6,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Kirk Snyder','PTS':3.8,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Steve Novak','PTS':3.9,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':5.2,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Steve Francis','PTS':5.5,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Gerald Green','PTS':6,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Mike James','PTS':6.5,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Luther Head','PTS':7.6,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Carl Landry','PTS':8.1,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Bobby Jackson','PTS':8.8,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Bonzi Wells','PTS':9.2,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Shane Battier','PTS':9.3,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Luis Scola','PTS':10.3,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Rafer Alston','PTS':13.1,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':21.6,'Cpl':20})
df3.loc[21] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':22,'Cpl':21})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2007/2008'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2007/2008")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1,1,3,3,3,3.6,3.8,3.9,5.2,5.5,6,6.5,7.6,8.1,8.8,9.2,9.3,10.3,13.1,21.6,22])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2007/2008'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2007/2008'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2008/2009")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Joey Dorsey','PTS':0.7,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':1.3,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Brian Cook','PTS':1.3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Dikembe Mutombo','PTS':1.6,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'James White','PTS':1.8,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Brent Barry','PTS':3.7,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Luther Head','PTS':4.8,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Shane Battier','PTS':7.3,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Kyle Lowry','PTS':7.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Carl Landry','PTS':9.2,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Von Wafer','PTS':9.7,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':11.2,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Rafer Alston','PTS':11.5,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Luis Scola','PTS':12.7,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':15.6,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Metta World Peace','PTS':17.1,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':19.7,'Cpl':17})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2008/2009'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2008/2009")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0.7,1.3,1.3,1.6,1.8,3.7,4.8,7.3,7.6,9.2,9.7,11.2,11.5,12.7,15.6,17.1,19.7])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2008/2009'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2008/2009'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2009/2010")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Hilton Armstrong','PTS':1.1,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Will Conroy','PTS':1.2,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Pops Mensah-Bonsu','PTS':1.3,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Brian Cook','PTS':1.5,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Joey Dorsey','PTS':1.6,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Mike Harris','PTS':3.1,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Tracy McGrady','PTS':3.2,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Jermaine Taylor','PTS':4.3,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':4.4,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Jared Jeffries','PTS':4.9,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Garrett Temple','PTS':5,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'David Andersen','PTS':5.8,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Jordan Hill','PTS':6.4,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Shane Battier','PTS':8,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Chase Budinger','PTS':8.9,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Kyle Lowry','PTS':9.1,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Trevor Ariza','PTS':14.9,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Carl Landry','PTS':16.1,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Luis Scola','PTS':16.4,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':19.6,'Cpl':20})
df3.loc[21] = pd.Series({'Joueur Houston Rockets':'Kevin Martin','PTS':21.3,'Cpl':21})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2009/2010'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2009/2010")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.1,1.2,1.3,1.5,1.6,3.1,3.2,4.3,4.4,4.9,5,5.8,6.4,8,8.9,9.1,14.9,16.1,16.4,19.6,21.3])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2009/2010'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2009/2010'):
    st.write(fig)


#En-tête
st.header("Moyenne du nombre de points de l'équipe des Houston Rockets de 2010 à 2018")

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2010/2011")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Hasheem Thabeet','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'DeMarre Carroll','PTS':0,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Jared Jeffries','PTS':1.6,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Mike Harris','PTS':2,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Ishmael Smith','PTS':2.6,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Terrence Williams','PTS':3.5,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Jermaine Taylor','PTS':4.9,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Jordan Hill','PTS':5.6,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Patrick Patterson','PTS':6.3,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Brad Miller','PTS':6.4,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Goran Dragic','PTS':7.7,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':7.9,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Courtney Lee','PTS':8.3,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Shane Battier','PTS':8.6,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Chase Budinger','PTS':9.8,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Ming Yao','PTS':10.2,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':11.6,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Kyle Lowry','PTS':13.5,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Luis Scola','PTS':18.3,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Kevin Martin','PTS':23.5,'Cpl':20})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2010/2011'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2010/2011")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,0,1.6,2,2.6,3.5,4.9,5.6,6.3,6.4,7.7,7.9,8.3,8.6,9.8,10.2,11.6,13.5,18.3,23.5])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2010/2011'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2010/2011'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2011/2012")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Hasheem Thabeet','PTS':1.2,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Greg Smith','PTS':1.8,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Marcus Morris','PTS':2.5,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Jeff Adrien','PTS':2.6,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Jonny Flynn','PTS':3.4,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Courtney Fortson','PTS':4.5,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Terrence Williams','PTS':4.5,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Earl Boykins','PTS':4.9,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Jordan Hill','PTS':5,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Marcus Camby','PTS':7.1,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Samuel Dalembert','PTS':7.5,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Patrick Patterson','PTS':7.7,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Chase Budinger','PTS':9.6,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Chandler Parsons','PTS':9.7,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Courtney Lee','PTS':11.4,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Goran Dragic','PTS':11.7,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Kyle Lowry','PTS':14.3,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Luis Scola','PTS':15.5,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Kevin Martin','PTS':17.1,'Cpl':19})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2011/2012'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2011/2012")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.2,1.8,2.5,2.6,3.4,4.5,4.5,4.9,5,7.1,7.5,7.7,9.6,9.7,11.4,11.7,14.3,15.5,17.1])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2011/2012'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2011/2012'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2012/2013")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Tim Ohlbrecht','PTS':1,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Scott Machado','PTS':1.3,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':1.4,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Cole Aldrich','PTS':1.7,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Daequan Cook','PTS':3.4,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'James Anderson','PTS':4,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Thomas Robinson','PTS':4.5,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Terrence Jones','PTS':5.5,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Patrick Beverley','PTS':5.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Donatas Motiejunas','PTS':5.7,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Greg Smith','PTS':6,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Francisco Garcia','PTS':6.4,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Toney Douglas','PTS':8.1,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Marcus Morris','PTS':8.6,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Omer Asik','PTS':10.1,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Carlos Delfino','PTS':10.6,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Patrick Patterson','PTS':11.6,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Jeremy Lin','PTS':13.4,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Chandler Parsons','PTS':15.5,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':25.9,'Cpl':20})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2012/2013'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2012/2013")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.0,1.3,1.4,1.7,3.4,4.0,4.5,5.5,5.6,5.7,6.0,6.4,8.1,8.6,10.1,10.6,11.6,13.4,15.5,25.9])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2012/2013'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2012/2013'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2013/2014")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Ronnie Brewer','PTS':0.3,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Greg Smith','PTS':3.5,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Robert Covington','PTS':4,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Josh Powell','PTS':4,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Isaiah Canaan','PTS':4.8,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Donatas Motiejunas','PTS':5.6,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Francisco Garcia','PTS':5.7,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Omer Asik','PTS':5.8,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Jordan Hamilton','PTS':6.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Omri Casspi','PTS':6.9,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Aaron Brooks','PTS':7.3,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Troy Daniels','PTS':8.4,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Patrick Beverley','PTS':10.2,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Terrence Jones','PTS':12.1,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Jeremy Lin','PTS':12.5,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Chandler Parsons','PTS':16.6,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Dwight Howard','PTS':18.3,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':25.4,'Cpl':18})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2013/2014'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2013/2014")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0.3,3.5,4,4,4.8,5.6,5.7,5.8,6.6,6.9,7.3,8.4,10.2,12.1,12.5,16.6,18.3,25.4])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2013/2014'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2013/2014'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2014/2015")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'K.J. McDaniels','PTS':1.1,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Nick Johnson','PTS':2.6,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Joey Dorsey','PTS':2.7,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Troy Daniels','PTS':2.9,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Pablo Prigioni','PTS':3,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Clint Capela','PTS':3.2,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Francisco Garcia','PTS':3.2,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Alexey Shved','PTS':3.2,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Tarik Black','PTS':4.2,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Kostas Papanikolaou','PTS':4.2,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Isaiah Canaan','PTS':6.4,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Jason Terry','PTS':7,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Patrick Beverley','PTS':10.1,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Terrence Jones','PTS':11.7,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Corey Brewer','PTS':11.9,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Josh Smith','PTS':12,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Donatas Motiejunas','PTS':12,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Trevor Ariza','PTS':12.8,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Dwight Howard','PTS':15.8,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':27.4,'Cpl':20})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2014/2015'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2014/2015")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([1.1,2.6,2.7,2.9,3,3.2,3.2,3.2,4.2,4.2,6.4,7,10.1,11.7,11.9,12,12,12.8,15.8,27.4])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2014/2015'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2014/2015'):
    st.write(fig)

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2015/2016")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Sam Dekker','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Chuck Hayes','PTS':2,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'K.J. McDaniels','PTS':2.6,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Andrew Goudelock','PTS':2.8,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Montrezl Harrell','PTS':3.6,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Ty Lawson','PTS':5.8,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Jason Terry','PTS':5.9,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Donatas Motiejunas','PTS':6.2,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Josh Smith','PTS':6.6,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Clint Capela','PTS':7,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Corey Brewer','PTS':7.2,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Terrence Jones','PTS':8.7,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Patrick Beverley','PTS':9.9,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Marcus Thornton','PTS':10,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Trevor Ariza','PTS':12.7,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Michael Beasley','PTS':12.8,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Dwight Howard','PTS':13.7,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':29,'Cpl':18})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2015/2016'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2015/2016")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,2,2.6,2.8,3.6,5.8,5.9,6.2,6.6,7,7.2,8.7,9.9,10,12.7,12.8,13.7,29])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2015/2016'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2015/2016'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2016/2017")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Isaiah Taylor','PTS':0.8,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Kyle Wiltjer','PTS':0.9,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Tyler Ennis','PTS':1.9,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Bobby Brown','PTS':2.5,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'K.J. McDaniels','PTS':2.7,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Chinanu Onuaku','PTS':2.8,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Corey Brewer','PTS':4.2,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Sam Dekker','PTS':6.5,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Montrezl Harrell','PTS':9.1,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'Nene Hilario','PTS':9.1,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Patrick Beverley','PTS':9.5,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Troy Williams','PTS':9.7,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Trevor Ariza','PTS':11.7,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'Clint Capela','PTS':12.6,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Ryan Anderson','PTS':13.6,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Lou Williams','PTS':14.8,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Eric Gordon','PTS':16.2,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':29.1,'Cpl':18})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2016/2017'):
    st.write(df3)


#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2016/2017")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0.8,0.9,1.9,2.5,2.7,2.8,4.2,6.5,9.1,9.1,9.5,9.7,11.7,12.6,13.6,14.8,16.2,29.1])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2016/2017'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2016/2017'):
    st.write(fig)


#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des Houston Rockets pour la saison 2017/2018")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df3 = pd.DataFrame(columns=['Joueur Houston Rockets','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])

df3.loc[1] = pd.Series({'Joueur Houston Rockets':'Isaiah Canaan','PTS':0,'Cpl':1})
df3.loc[2] = pd.Series({'Joueur Houston Rockets':'Demetrius Jackson','PTS':0.7,'Cpl':2})
df3.loc[3] = pd.Series({'Joueur Houston Rockets':'Qi Zhou','PTS':1.2,'Cpl':3})
df3.loc[4] = pd.Series({'Joueur Houston Rockets':'Troy Williams','PTS':1.3,'Cpl':4})
df3.loc[5] = pd.Series({'Joueur Houston Rockets':'Markel Brown','PTS':1.3,'Cpl':5})
df3.loc[6] = pd.Series({'Joueur Houston Rockets':'Tim Quarterman','PTS':1.3,'Cpl':6})
df3.loc[7] = pd.Series({'Joueur Houston Rockets':'Briante Weber','PTS':2,'Cpl':7})
df3.loc[8] = pd.Series({'Joueur Houston Rockets':'Bobby Brown','PTS':2.5,'Cpl':8})
df3.loc[9] = pd.Series({'Joueur Houston Rockets':'Tarik Black','PTS':3.5,'Cpl':9})
df3.loc[10] = pd.Series({'Joueur Houston Rockets':'R.J. Hunter','PTS':3.8,'Cpl':10})
df3.loc[11] = pd.Series({'Joueur Houston Rockets':'Chinanu Onuaku','PTS':4,'Cpl':11})
df3.loc[12] = pd.Series({'Joueur Houston Rockets':'Brandan Wright','PTS':4,'Cpl':12})
df3.loc[13] = pd.Series({'Joueur Houston Rockets':'Joe Johnson','PTS':6,'Cpl':13})
df3.loc[14] = pd.Series({'Joueur Houston Rockets':'P.J. Tucker','PTS':6.1,'Cpl':14})
df3.loc[15] = pd.Series({'Joueur Houston Rockets':'Nene Hilario','PTS':6.5,'Cpl':15})
df3.loc[16] = pd.Series({'Joueur Houston Rockets':'Luc Mbah A Moute','PTS':7.5,'Cpl':16})
df3.loc[17] = pd.Series({'Joueur Houston Rockets':'Aaron Jackson','PTS':8,'Cpl':17})
df3.loc[18] = pd.Series({'Joueur Houston Rockets':'Ryan Anderson','PTS':9.3,'Cpl':18})
df3.loc[19] = pd.Series({'Joueur Houston Rockets':'Trevor Ariza','PTS':11.7,'Cpl':19})
df3.loc[20] = pd.Series({'Joueur Houston Rockets':'Gerald Green','PTS':12.1,'Cpl':20})
df3.loc[21] = pd.Series({'Joueur Houston Rockets':'Clint Capela','PTS':13.9,'Cpl':21})
df3.loc[22] = pd.Series({'Joueur Houston Rockets':'Eric Gordon','PTS':18,'Cpl':22})
df3.loc[23] = pd.Series({'Joueur Houston Rockets':'Chris Paul','PTS':18.6,'Cpl':23})
df3.loc[24] = pd.Series({'Joueur Houston Rockets':'James Harden','PTS':30.4,'Cpl':24})

df3["CPTS"] = np.cumsum(df3["PTS"])

df3["Cpl%"] = df3["Cpl"].apply(lambda x : ((x/df3.loc[len(df3),"Cpl"])*100))

df3["CPTS%"] = df3["CPTS"].apply(lambda x : (x/df3.loc[len(df3),"CPTS"])*100)

if st.checkbox('Show dataframe2 2017/2018'):
    st.write(df3)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des Houston Rockets pour la saison 2017/2018")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Houston Rockets

fig = plt.figure(figsize=(10, 6))

# assurez-vous que votre arr est d’abord trié des valeurs les plus basses aux plus élevées!
arr = np.array([0,0.7,1.2,1.3,1.3,1.3,2,2.5,3.5,3.8,4,4,6,6.1,6.5,7.5,8,9.3,11.7,12.1,13.9,18,18.6,30.4])

def gini(arr):
    count = arr.size
    coefficient = 2 / count
    indexes = np.arange(1, count + 1)
    weighted_sum = (indexes * arr).sum()
    total = arr.sum()
    constant = (count + 1) / count
    return coefficient * weighted_sum / total - constant

def lorenz(arr):
    # cela divise la somme du préfixe par la somme totale
    # cela garantit que toutes les valeurs sont comprises entre 0 et 1,0
    scaled_prefix_sum = arr.cumsum() / arr.sum()
    # cela ajoute la valeur 0 (parce que 0% de toutes les personnes ont 0% de toute la richesse)
    return np.insert(scaled_prefix_sum, 0, 0)

# afficher l’index de gini!
if st.checkbox('Show coefficient de Gini2 2017/2018'):
  st.write(gini(arr))

lorenz_curve = lorenz(arr)

# nous avons besoin que les valeurs X soient comprises entre 0,0 et 1,0
plt.plot(np.linspace(0.0, 1.0, lorenz_curve.size), lorenz_curve)
# tracer la courbe d’égalité parfaite en ligne droite
plt.plot([0,1], [0,1])

if st.checkbox('Show courbe de Lorenz2 2017/2018'):
    st.write(fig)



# Constitution des DataFrames

st.header("Constitution des DataFrames")

#Sous en-tête
st.subheader("Nomination des colonnes")


#Ecrire texte
st.write("Gini: Coefficient qui détermine l’homogénéité ou l’hétérogénéité d’une équipe sportive")

st.write("1_Marqueur : Le meilleur marqueur de l’équipe en question")

st.write("Moyenne_PTS_1_marqueur : Moyenne de points du meilleur marqueur")

st.write("Gini_sans_1_marqueur : Coefficient Gini de toute l’équipe sans le meilleur marqueur. Permet de voir si le taulier de l’équipe la « déséquilibre» en terme de moyenne de points.")
st.write("Plus l’écart sera significatif avec le coefficient Gini de l’équipe, plus l’équipe dépend du meilleur marqueur")
st.write("Par rapport à notre étude, un écart supérieur à 0.3 signifie que l’écart commence à être conséquent")

st.write("Gini_des_trois_meilleurs_marqueurs: Coefficients Gini des trois meilleurs marqueurs de l’équipe.Permet de voir l’homogénéité ou l’hétérogénéité des trois meilleurs marqueurs de l’équipe en terme de moyenne de points.")
st.write("Plus ce nombre se rapproche de zéro, plus la moyenne des points sera correctement répartie entre les trois joueurs.")
st.write("Par rapport à notre étude, un coefficient supérieur à 1 ou qui se rapproche très fortement de 1 signifie un début d’hétérogénéité.")

st.write("2_marqueur: Le deuxième meilleur marqueur de l’équipe")

st.write("3_marqueur: Le troisième meilleur marqueur de l’équipe")


#Sous en-tête

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


st.subheader("Les quatre DataFrames de notre étude")

st.subheader("DataFrame des San Antonio Spurs des années 2000/2010")

df4 = pd.read_csv('DataFrame_San_Antonio_Spurs_2000_2010.csv')

df4


st.subheader("DataFrame des San Antonio Spurs des années 2010/2018")

df5 = pd.read_csv('DataFrame_San_Antonio_Spurs_2010_2018.csv')

df5



st.subheader("DataFrame des Houston Rockets des années 2000/2010")

df6 = pd.read_csv('DataFrame_Houston_Rockets_2000_2010.csv')

df6



st.subheader("DataFrame des Houston Rockets des années 2010/2018")

df7 = pd.read_csv('DataFrame_Houston_Rockets_2010_2018.csv')

df7


st.subheader("coefficients de Gini de l’équipe des San Antonio Spurs et des Houston Rockets de 2000 à 2010")

fig = plt.figure(figsize=(10, 6))
barWidth = 0.25

x1 = range(10)

x2 = [r + barWidth for r in range(10)]


plt.bar(x1, df4["Gini"], width = 0.2, label = 'Gini Spurs')
plt.bar(x2, df6["Gini"], width = 0.2, label = 'Gini Rockets')

plt.xticks(rotation='vertical')
plt.xticks([0,1,2,3,4,5,6,7,8,9], ['2000/2001', '2001/2002', '2002/2003', '2003/2004', '2004/2005', '2005/2006', '2006/2007', '2007/2008', '2008/2009', '2009/2010'])
plt.legend()
st.pyplot(fig);


st.subheader("Les trois meilleurs marqueurs de l’équipe des San Antonio Spurs de 2000 à 2010(nombre de présences aux trois premières places)")

df10 = df4[['1_Marqueur', '2_marqueur', '3_marqueur']].apply(pd.value_counts)

df10 = df10.fillna(0)

df10

df10 = df10.apply(np.sum, axis = 1)

df10 = df10.sort_values(ascending = False)

df10


st.subheader("Les trois meilleurs marqueurs de l’équipe des Houston Rockets de 2000 à 2010(nombre de présences aux trois premières places)")

df12 = df6[['1_Marqueur', '2_marqueur', '3_marqueur']].apply(pd.value_counts)

df12 = df12.fillna(0)

df12

df12 = df12.apply(np.sum, axis = 1)

df12 = df12.sort_values(ascending = False)

df12


st.subheader("coefficients de Gini de l’équipe des San Antonio Spurs et des Houston Rockets de 2010 à 2018")

fig = plt.figure(figsize=(10, 6))
barWidth = 0.25

x1 = range(8)

x2 = [r + barWidth for r in range(8)]


plt.bar(x1, df5["Gini"], width = 0.2, label = 'Gini Spurs')
plt.bar(x2, df7["Gini"], width = 0.2, label = 'Gini Rockets')

plt.xticks(rotation='vertical')
plt.xticks([0,1,2,3,4,5,6,7], ['2010/2011', '2011/2012', '2012/2013', '2013/2014', '2014/2015', '2015/2006', '2016/2017', '2017/2018'])
plt.legend()
st.pyplot(fig);

st.subheader("Les trois meilleurs marqueurs de l’équipe des San Antonio Spurs de 2010 à 2018(nombre de présences aux trois premières places)")

df15 = df5[['1_Marqueur', '2_marqueur', '3_marqueur']].apply(pd.value_counts)

df15 = df15.fillna(0)

df15

df15 = df15.apply(np.sum, axis = 1)

df15 = df15.sort_values(ascending = False)

df15

st.subheader("Les trois meilleurs marqueurs de l’équipe des Houston Rockets de 2000 à 2010(nombre de présences aux trois premières places)")

df17 = df7[['1_Marqueur', '2_marqueur', '3_marqueur']].apply(pd.value_counts)

df17 = df17.fillna(0)

df17

df17 = df17.apply(np.sum, axis = 1)

df17 = df17.sort_values(ascending = False)

df17

   


Overwriting app.py


# Nouvelle section

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok
!ngrok authtoken 2KsGKlEb2d6sD1HFYYBc885yejd_5zfTGyd4MeHvaWAGQozHa
public_url = ngrok.connect(port='8501')
print(public_url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
http://83c7-35-197-105-211.ngrok.io


In [ ]:
ngrok.kill()